# AI Generated Images for your Roboflow Project using Stable Diffusion

[Stable Diffusion](https://stability.ai/blog/stable-diffusion-public-release) is a deep learning, text-to-image model released in 2022. It is primarily used to generate detailed images conditioned on text descriptions, though it can also be applied to other tasks such as inpainting, outpainting, and generating image-to-image translations guided by a text prompt. In this notebook, we'll use Stable Diffusion to generate images for your Computer Vision project, and push those images into your [Roboflow](https://blog.roboflow.com/synthetic-data-with-stable-diffusion-a-guide/) project for annotating. We will be using [Amazon SageMaker Studio Lab](https://studiolab.sagemaker.aws/). At the end, we'll have quality, AI generated, representative data to further enrich your dataset and strengthen your model.

Many thanks to the CompVis group LMU Munich, Runway and Stability AI for releasing [the code](https://github.com/CompVis/stable-diffusion).

## **Steps Covered in this Tutorial**

To generate our images we will cover the following:

* Install Stable Diffusion dependencies
* Getting latest model version - runwayml/stable-diffusion-v1-5 - from Hugging Face hub
* Create function to generate images
* Generate images based on your prompt of choice
* Upload images to your Roboflow project using Roboflow PIP package
* Begin Annotating

## Installing Roboflow and other dependencies

We'll be using [Roboflow](https://roboflow.com/?ref=studiolab) to push our images up to after we have generated them for annotating (and, optionally, to use the [Roboflow Annotate tool](https://roboflow.com/annotate).

The [`roboflow` pip package](https://blog.roboflow.com/pip-install-roboflow/) will allow us to upload our batch of generated images.

In [4]:
%%sh
pip install -q --upgrade pip
pip install -q --upgrade diffusers transformers scipy ftfy huggingface_hub roboflow

## Authenticating with the Hugging Face Hub

We'll be using [Hugging Face](https://huggingface.co/) to pull down our Stable Diffusion model, so we must authenticate using our [Hugging Face Access Token](https://huggingface.co/docs/hub/security-tokens) 

When we run the below cell, we enter our token, click login and we are authenticated.

_***Note: You don't get a confirmation of token accepted once you click login. You can however confirm you are authenticated by looking at the SageMaker Studio Lab logs in the terminal at the bottom of your screen.***_

In [8]:
from huggingface_hub import notebook_login

# Required to get access to stable diffusion model
notebook_login()

## Accepting License Terms

Before we load this model from the Hugging Face Hub, we have to make sure that we accept the license of the runwayml/stable-diffusion-v1-5 project. You can accept the license by clicking on the Agree and access repository button on the [model page](https://huggingface.co/runwayml/stable-diffusion-v1-5).

## Using the Hugging Face StableDiffusionPipeline Class

Here we will create our [Hugging Face Stable Diffusion pipeline](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion), as well as ensure we are running on cuda. Hugging Face pipelines are an easy way to use your Hugging Face models for [inference](https://huggingface.co/docs/transformers/main_classes/pipelines)

In [9]:
import torch
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16, revision="fp16"
)

pipeline = pipeline.to("cuda")

## Creating our Generate Images Function

After we have created our pipeline, we will create our function to generate images. Here we define some parameters:

* prompt = The prompt used to generate your images
* num_images_to_generate = Total number of images to generate
* num_images_per_prompt = The number of images to generate in one iteration
* guidance_scale = The guidance scale defines how much freedom you want to give the model. Higher guidance scale encourages the model to generate images that are closely linked to the text prompt, usually at the expense of lower image quality. [Guidance scale as defined in Classifier-Free Diffusion Guidance](https://arxiv.org/abs/2207.12598)
* output_dir = This is the location you want to save the images to (The location will be created when creating the images)
* display_images = Defines if you want to display the images inline after creation

You can read more about the parameters associated with the Stable Diffusion pipeline [here](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion)

In the function we iterate through all images created, based on our total defined images

In [11]:
import os

from IPython.display import Image, display


def generate_images(
    prompt,
    num_images_to_generate,
    num_images_per_prompt=4,
    guidance_scale=8,
    output_dir="generated_images",
    display_images=False,
):

    num_iterations = num_images_to_generate // num_images_per_prompt
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_iterations):
        images = pipeline(
            prompt, num_images_per_prompt=num_images_per_prompt, guidance_scale=guidance_scale
        )
        for idx, image in enumerate(images.images):
            image_name = f"{output_dir}/image_{(i*num_images_per_prompt)+idx}.png"
            image.save(image_name)
            if display_images:
                display(Image(filename=image_name, width=128, height=128))

In [13]:
# 1000 images takes 2-3 hours on a SageMaker Studio Lab GPU instance. 
# You can adjust the total image number below

generate_images("aerial view of cattle", 12, guidance_scale=4, display_images=True)



## Sign up for a free Roboflow Account
_***Note: If you already have a Roboflow account and project created, you can skip the below steps and go right to pushing your generated images up to your Roboflow project.***_

### Sign up for a free Roboflow account

[Roboflow](https://roboflow.com/?ref=studiolab) is an end-to-end computer vision platform. It helps you [create](https://docs.roboflow.com/quick-start?ref=studiolab), [understand](https://blog.roboflow.com/dataset-search/?ref=studiolab), and [use](https://docs.roboflow.com/exporting-data?ref=studiolab) image datasets to train and deploy custom models.

Roboflow strives to be broadly interoperable and can import and export object detection datasets in [dozens of formats](https://roboflow.com/formats?ref=studiolab). They maintain [training notebooks](https://models.roboflow.com/?ref=studiolab) (like this one) for many state of the art computer vision models, and also offer [AutoML training](https://docs.roboflow.com/train?ref=studiolab) which can be useful for [prototyping](https://blog.roboflow.com/deploy-tab/?ref=studiolab), [model assisted labeling](https://roboflow.com/annotate?ref=studiolab), and even [deploying to a wide range of targets and edge devices](https://roboflow.com/deploy?ref=studiolab).

In this tutorial, we will use Roboflow to annotate a custom dataset and export it for use with YOLOv7 in this notebook. But we encourage you to explore [its other features](https://roboflow.com/features?ref=studiolab) as well.

### Step 2: Create a Public Workspace

Roboflow offers a [generous free tier](https://roboflow.com/pricing?ref=studiolab) if your data can be shared publicly with others on [Roboflow Universe](https://universe.roboflow.com/?ref=studiolab). There are also paid plans available for private data.

For this tutorial you'll need to create a Public workspace. Be sure to give it a good name; it will serve as your Universe username where you can showcase your work.

<div><img src="https://i.imgur.com/zfE5MZL.png" style="max-width: 600px;"></div>
    
### Step 3: Create a Project

Then, create an `Object Detection` project (be sure to give it a descriptive name, and fill in the `What will your model predict?` section since they will make your project more understandable and be pulled in via the API later, and can serve as helpful metadata later for advanced use-cases like automated prompt engineering for zero-shot models).

<div><img src="https://i.imgur.com/O2xDyxQ.png" style="max-width: 500px;"></div>


## Push Generated Data to your Roboflow Project

### Upload your Images to Roboflow via PIP package

Here we will use the [Upload API](https://docs.roboflow.com/adding-data/upload-api?ref=studiolab) to push your images up to your Roboflow project. Pushing your images programatically instead of the web UI is especially important in the world of [Active Learning](https://docs.roboflow.com/python/active-learning).

You also have the ability to push those images up to your S3 bucket before sending over to Roboflow [load images from an S3 bucket](https://blog.roboflow.com/how-to-use-s3-computer-vision-pipeline/?ref=studiolab).

_**Note:** To get good, generalizable results you will need lots of images covering a wide variety of situations and edge cases. Exactly how many images you need [depends on a wide variety of factors](https://blog.roboflow.com/images-train-model/?ref=studiolab), but we recommend starting out with at least 200 for most use-cases. If you need more images, try sourcing from open source datasets on [Roboflow Universe](https://universe.roboflow.com/?ref=studiolab) with images similar to yours._


Set HOME path

In [14]:
import os
HOME = os.getcwd()

Once your images are generated, you call the [Roboflow Upload API](https://docs.roboflow.com/adding-data/upload-api) to push our images into our Roboflow project. 

In the cell below, you should substitute your API key and project name with a project and API that you create in Roboflow. 

To get your API key, you can follow our tutorial on [retrieving an API key from the Roboflow dashboard](https://docs.roboflow.com/rest-api#obtaining-your-api-key).

In [15]:
from roboflow import Roboflow
import glob
import os

# glob params
image_dir = os.path.join(HOME, "generated_images", "")
file_extension_type = ".png"

# roboflow pip params
rf = Roboflow(api_key="YOUR_API_KEY")
upload_project = rf.workspace().project("YOUR_PROJECT_NAME")

# glob images
image_glob = glob.glob(image_dir + '/*' + file_extension_type)

# perform upload
for image in image_glob:
    upload_project.upload(image, num_retry_uploads=3)
    print("*** Processing image [" + str(len(image_glob)) + "] - " + image + " ***")

## Check Images

Once the above cell is complete, all images should now be in your project, under the annotate section with the designation of _unassigned <> PIP Package Upload_.

<div><img src="https://i.imgur.com/fqAlyM2.png" style="max-width: 700px;"></div>

When you click in the _PIP Package Upload_ batch, you will be able to see all your generated data

<div><img src="https://i.imgur.com/EJxLTLq.png" style="max-width: 700px;"></div>

## Roboflow Annotate

Now, we'll use [Roboflow Annotate](https://roboflow.com/annotate?ref=studiolab) to create annotations that will teach our model what we're trying to detect in our images. Since your model will learn to mimic your annotations, it's important that you give some thought to how you label your images ahead of time. We've compiled a list of [best practices to consider when labeling images](https://blog.roboflow.com/tips-for-how-to-label-images/?ref=studiolab).

## Further Enhancements 

You can take these text-to-image capabilites a step further, further simplify this workflow and really take advantage of [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). If you locate an open source model on [Roboflow Universe](https://universe.roboflow.com/) that meets your models criteria, you can use this model to help do automated annotations.

Roboflow Universe has over 15k pre-trained and fine tuned models for use. Once you locate a useful model, you can call that model to get predictions. In the outout of that prediction is the predicted bounding box coordinates for your AI generated images.

You can then setup a workflow which calls the Universe model, takes all predictions with a confidence of over 50% and use the output of the prediction to auto label your images and get annotations. You will then upload all images and corresponding annotations into your Roboflow project via the API. 

This workflow would greatly reduce the manual efforts of annotating your generated images. I can't wait to see how people take this notebook further! 